In [93]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from toolsets.file_io import readin_peak_list
import toolsets.spectra_operations as so
from toolsets.search import string_search, quick_search_values

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
peak_list_dir = '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230713_2564_AX_P4_Active/peak_list'
useful_columns = ['RT (min)', 'Precursor m/z', 'Height', 'MS1 isotopes', 'MSMS spectrum']
filtered_dataset = pd.DataFrame()
reference_file = []
for root, dirs, files in os.walk(peak_list_dir):

    for file in tqdm(files):
        filename = file.split('.')[0]
        features_temp = readin_peak_list(os.path.join(peak_list_dir, file))
        features_temp = features_temp[~features_temp['MSMS spectrum'].isnull()]
        features_temp =string_search(features_temp, 'Isotope', 'M + 0')
        features_temp = quick_search_values(features_temp, 'RT (min)', 4, 6, ifsorted=False)
        features_temp = quick_search_values(features_temp, 'Precursor m/z', 250, 430, ifsorted=False)
        features_temp['reference_file']=filename
        filtered_dataset = pd.concat([filtered_dataset, features_temp], ignore_index=True)
        # print(file)
        # if file.endswith('.txt') and file != '2564_AX_P4_X21-23_X5.txt':


            # break
            # feature_subsets = quick_search_values(features_temp,'rt', value_start=4, value_end=6, ifsorted=False)
            # feature_4_6 = pd.concat([feature_4_6, feature_subsets], ignore_index=True)

100%|██████████| 14/14 [00:11<00:00,  1.17it/s]


In [96]:
filtered_dataset['Height'].quantile(0.75)

2074113.25

In [97]:
filtered_dataset_75 = filtered_dataset[filtered_dataset['Height']>filtered_dataset['Height'].quantile(0.75)]

In [98]:
filtered_dataset_75.sort_values(by = 'Precursor m/z', ascending=True, inplace=True, ignore_index=True)

In [99]:
ms1 = []
msms = []
for index, row in filtered_dataset_75.iterrows():
    ms1_raw = so.convert_msdial_to_string(row['MS1 isotopes'])
    ms1.append(so.remove_zero_ions(ms1_raw))
    # ms1.append(so.convert_msdial_to_string(row['MS1 isotopes']))
    msms.append((so.convert_msdial_to_string(row['MSMS spectrum'])))
filtered_dataset_75['ms1']=ms1

filtered_dataset_75['msms']=msms

In [100]:
filtered_dataset_75.to_csv('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230713_2564_AX_P4_Active/sirius_compitable/all_20230726.csv', index=False)

In [101]:
from toolsets.file_io import export_mgf_sirius

In [102]:
# os.mkdir('/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230713_2564_AX_P4_Active/sirius_compitable_updated')

FileExistsError: [Errno 17] File exists: '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230713_2564_AX_P4_Active/sirius_compitable_updated'

In [104]:
for index, row in tqdm(filtered_dataset_75.iterrows(), total = len(filtered_dataset_75)):
    export_mgf_sirius(row, '/Volumes/Natural Product Chemistry/Working Space/Fanzhou/20230713_2564_AX_P4_Active/sirius_compitable_updated')

100%|██████████| 320/320 [00:35<00:00,  9.06it/s]


In [40]:
import toolsets.spectra_operations as so